In [11]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd
# import os

In [6]:
#inputs
dataGen = pd.read_excel('inputs.xlsx', sheet_name='gen')
dataLoad = pd.read_excel('inputs.xlsx', sheet_name='load')
Ng = len(dataGen)

In [7]:
#model
model = pyo.ConcreteModel()

model.Pg = pyo.Var(range(Ng), bounds=(0,None))
Pg = model.Pg



In [9]:
#constraints
pg_sum = sum([Pg[g] for g in dataGen.id])
model.balance = pyo.Constraint(expr = pg_sum == sum(dataLoad.value))

model.cond = pyo.Constraint(expr = Pg[0]+Pg[3] >= dataLoad.value[0])
model.limits = pyo.ConstraintList()
for g in dataGen.id:
    model.limits.add(expr = Pg[g] <= dataGen.limit[g])
    


    'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a
    new Component (type=<class
    'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [10]:
#objFund

cost_sum = sum([Pg[g]*dataGen.cost[g] for g in dataGen.id])
model.obj = pyo.Objective(expr = cost_sum)

opt = SolverFactory('gurobi')
results = opt.solve(model)

dataGen['Pg'] = [pyo.value(Pg[g]) for g in dataGen.id]
print(dataGen)





   id  limit  cost    Pg
0   0     20  0.10  20.0
1   1     10  0.05  10.0
2   2     40  0.30  35.0
3   3     50  0.40  30.0
4   4      5  0.01   5.0
